[Reference](https://towardsdatascience.com/freezing-a-keras-model-c2e26cb84a38)

In [1]:
#import keras modules in tensorflow implementation
from tensorflow import keras
from tensorflow.keras.layers import Convolution2D, MaxPool2D, Flatten, Dense

import numpy as np
import tensorflow as tf

def CNN(input_layer):
  '''
  defines the layers of the model
  input_layer - pass input layer name
  '''
  conv1 = Convolution2D(16, 2, padding = 'same', activation = 'relu')(input_layer)
  pool1 = MaxPool2D(pool_size = 2)(conv1)
  
  conv2 = Convolution2D(32, 2, padding = 'same', activation = 'relu')(pool1)
  pool2 = MaxPool2D(pool_size = 2)(conv2)
    
  flat = Flatten()(pool2)
  dense = Dense(128, activation = 'relu')(flat)
    
  output = Dense(10, activation  = 'softmax', name = "output_node")(dense)
  return output

In [2]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

#define input layer
inpt = Input(shape = (28,28,1), name = "input_node")

#call the model
logits = CNN(inpt)

#define model
model = Model(inpt,logits)

#compile the model
model.compile(optimizer = keras.optimizers.Adam(lr = 0.0001), \
              loss = 'categorical_crossentropy', metrics = ['accuracy'])

#convert to an Estimator
# the model_dir states where the graph and checkpoint files will be saved to
estimator_model = tf.keras.estimator.model_to_estimator(keras_model = model, \
                                                        model_dir = './Keras_MNIST')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/backend.py:401: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': './Keras_MNIST', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
def input_function(features,labels=None,shuffle=False):
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"input_node": features},
        y=labels,
        shuffle=shuffle,
        batch_size = 5,
        num_epochs = 1
    )
    return input_fn
  
estimator_model.train(input_fn = input_function(X_train,y_train,True))

In [5]:
#since we are working in TensorFlow we define placeholder layers
X = tf.placeholder(tf.float32, [None, 28,28,1], name = "input_node")
y = tf.placeholder(tf.float32, [None, 10])

#call the model
logits = mod(X)

# Freezing the model


In [6]:
def freeze_graph(model_dir, output_node_names):
  """Extract the sub graph defined by the output nodes and convert 
  all its variables into constant 
  Args:
      model_dir: the root folder containing the checkpoint state file
      output_node_names: a string, containing all the output node's names, 
                          comma separated
                        """
  if not tf.gfile.Exists(model_dir):
    raise AssertionError(
      "Export directory doesn't exists. Please specify an export "
      "directory: %s" % model_dir)

  if not output_node_names:
    print("You need to supply the name of a node to --output_node_names.")
    return -1

  # We retrieve our checkpoint fullpath
  checkpoint = tf.train.get_checkpoint_state(model_dir)
  input_checkpoint = checkpoint.model_checkpoint_path
    
  # We precise the file fullname of our freezed graph
  absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
  output_graph = absolute_model_dir + "/frozen_model.pb"

  # We clear devices to allow TensorFlow to control on which device it will load operations
  clear_devices = True

  # We start a session using a temporary fresh Graph
  with tf.Session(graph=tf.Graph()) as sess:
    # We import the meta graph in the current default Graph
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    # We restore the weights
    saver.restore(sess, input_checkpoint)

    # We use a built-in TF helper to export variables to constants
    output_graph_def = tf.graph_util.convert_variables_to_constants(
      sess, # The session is used to retrieve the weights
      tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
      output_node_names.split(",") # The output node names are used to select the usefull nodes
    ) 

    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile(output_graph, "wb") as f:
      f.write(output_graph_def.SerializeToString())
    print("%d ops in the final graph." % len(output_graph_def.node))

  return output_graph_def